<a href="https://colab.research.google.com/github/sillHD/CuadernosSyS/blob/main/Parcial1_de_SyS_2024_2_Ismael_Cortes_Ramirez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Frecuencia y periodo de muestreo
Fs = 5000
Ts = 1/Fs

# Se crea un vector con los valores de las amplitudes
A = [0.3,0.6,0.1]

# Se crea un vector con los valores de la velociades angulares
Wo = [1000*np.pi,2000*np.pi,11000*np.pi]

# Se crea un vector para las fases
phi = [-np.pi/4,0,-np.pi]

# se crea un vector vacio el cual almacenara las frecuencias de cada seccion de la funcion, despues se usa un ciclo que  utiliza el vector Wo para calcular las frecuencias
F =[]
for i in range(len(Wo)):
  F.append(Wo[i]/(2*np.pi))

# se crea un vector para definir los valores de t, debemos tener en cuenta que se nos pidio graficar al menos 3 ciclos, por lo cual necesitaremos la parte de la funcion que tiene la frencuencia menor para asi calcular el periodo maximo de la funcion
T_max  = 1/F[0]
t = np.linspace(0,3*T_max,5000)

# Se crea una variable que sera la funcion que se nos dio del ejercicio, para poder llenarla se usa un ciclo que va usando los vectores A y Wo para armar cada seccion de la funcion y agregarla a x
x = 0
for i in range(len(A)):
  if i%2 == 0:
    x += (A[i]*np.cos(Wo[i]*t + phi[i]))
  else:
    x += (A[i]*np.sin(Wo[i]*t + phi[i]))

# Proceso de discretizacion
tv = np.arange(0,3*T_max,Ts)
nbits  = 5
xv = 0
rmin = -0.75
rmax = 1
ve  = np.linspace(rmin,rmax,2**nbits)

for i in range(len(A)):
  if i%2 == 0:
    xv += (A[i]*np.cos(Wo[i]*tv + phi[i]))
  else:
    xv += (A[i]*np.sin(Wo[i]*tv + phi[i]))

# esta seccion grafica la funcion continua
plt.figure(figsize=(12, 9)) # hace que la grafica sea mas ancha
plt.subplot(3, 1, 1)
plt.plot(t,x, label="Señal continua", color='red')
plt.title("Señal Continua")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()
plt.xlim(-0.00005, 3*T_max+0.0001) # Ajusta el límite inferior del eje x a 0

# esta seccion grafica la señal discreta
plt.subplot(3, 1, 2)
plt.plot(tv,xv, color='blue', label="Señal discreta")
plt.title("Señal Discreta")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()
plt.xlim(-0.00005, 3*T_max+0.0001) # Ajusta el límite inferior del eje x a 0

# Esta seccion grafica la señal digital
plt.subplot(3, 1, 3)
for v in ve:
  plt.plot(tv,v*np.ones(len(tv))) # grafica los 32 estados que se obtuvieron de los 5 bits
plt.plot(tv,xv, color='green')
plt.stem(tv, xv, linefmt='g-', markerfmt='go', basefmt=' ', label="Señal Digital") # genera  un grafico de tallo con lineas verticales
plt.title("Señal Digital")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()
plt.xlim(-0.00005, 3*T_max) # Ajusta el límite inferior del eje x a 0


plt.tight_layout()
plt.show()

si comparamos las dos graficas podemos notar que se presenta un aliasing en la grafica discreta debido a que la frecuencia de muestreo no cumple el teorema de nyquist,  por lo cual procederemos implementar un conversor adecuado para la señal estudiada, para esto usaremos como base el codigo anterior, con el ligero cambio de que estableceremos la frecuencia de muestreo y el numero de bits como variables de entrada

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Se crea un vector con los valores de las amplitudes
A = [0.3,0.6,0.1]

# Se crea un vector con los valores de la velociades angulares
Wo = [1000*np.pi,2000*np.pi,11000*np.pi]

# Se crea un vector para las fases
phi = [-np.pi/4,0,-np.pi]

# se crea un vector vacio el cual almacenara las frecuencias de cada seccion de la funcion, despues se usa un ciclo que  utiliza el vector Wo para calcular las frecuencias
F =[]
for i in range(len(Wo)):
  F.append(Wo[i]/(2*np.pi))
F_max =  max(F)

# Variables de entrada
Fs = int(input("Ingrese la frecuencia de muestreo:\n "))
while Fs < 2*F_max:
  Fs = int(input(f"La frecuencia ingresada no cumple con el teorema de Nyquist, recuerde que para este caso la frecuencia de muestreo(Fs) debe ser mayor o igual a {2*round(F_max)}Hz\n" ))
Ts = 1/Fs
nbits  = int(input("Ingrese el numero de bits:\n "))
# se crea un vector para definir los valores de t, debemos tener en cuenta que se nos pidio graficar al menos 3 ciclos, por lo cual necesitaremos la parte de la funcion que tiene la frencuencia menor para asi calcular el periodo maximo de la funcion
T_max  = 1/min(F)
t = np.linspace(0,3*T_max,5000)

# Se crea una variable que sera la funcion que se nos dio del ejercicio, para poder llenarla se usa un ciclo que va usando los vectores A y Wo para armar cada seccion de la funcion y agregarla a x
x = 0
for i in range(len(A)):
  if i%2 == 0:
    x += (A[i]*np.cos(Wo[i]*t + phi[i]))
  else:
    x += (A[i]*np.sin(Wo[i]*t + phi[i]))

# Proceso de discretizacion
tv = np.arange(0,3*T_max,Ts)
xv = 0
rmin = -0.75
rmax = 1
ve  = np.linspace(rmin,rmax,2**nbits)

for i in range(len(A)):
  if i%2 == 0:
    xv += (A[i]*np.cos(Wo[i]*tv + phi[i]))
  else:
    xv += (A[i]*np.sin(Wo[i]*tv + phi[i]))

# esta seccion grafica la funcion continua
plt.figure(figsize=(12, 9)) # hace que la grafica sea mas ancha
plt.subplot(3, 1, 1)
plt.plot(t,x, label="Señal continua", color='red')
plt.title("Señal Continua")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()
plt.xlim(-0.00005, 3*T_max+0.0001) # Ajusta el límite inferior del eje x a 0

# esta seccion grafica la señal discreta
plt.subplot(3, 1, 2)
plt.plot(tv,xv, color='blue', label="Señal discreta")
plt.title("Señal Discreta")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()
plt.xlim(-0.00005, 3*T_max+0.0001) # Ajusta el límite inferior del eje x a 0

# Esta seccion grafica la señal digital
plt.subplot(3, 1, 3)
for v in ve:
  plt.plot(tv,v*np.ones(len(tv))) # grafica los 32 estados que se obtuvieron de los 5 bits
plt.plot(tv,xv, color='green')
plt.stem(tv, xv, linefmt='g-', markerfmt='go', basefmt=' ', label="Señal Digital") # genera  un grafico de tallo con lineas verticales
plt.title("Señal Digital")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.grid()
plt.legend()
plt.xlim(-0.00005, 3*T_max) # Ajusta el límite inferior del eje x a 0


plt.tight_layout()
plt.show()

# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)


# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()